In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

cuda = torch.device('cuda:1')
cuda

deathWoman = np.array(pd.read_excel('../../data/여성사망률추이.xlsx').set_index(['연도']).iloc[:43,:100])
deathMan = np.array(pd.read_excel('../../data/남성사망률추이.xlsx').set_index(['연도']).iloc[:43,:100])
dataOriDf = pd.read_excel('../../data/데이터정리3.xlsx')
ageDataOriDf = pd.read_excel('../../data/의사연령별분포.xlsx')
npAgeData = np.array(pd.read_excel('../../data/의사연령별분포.xlsx'))

#1977년 합격자부터 2019년 합격자
dataDf = dataOriDf[['년도','의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]
dataDf = dataDf[27:70]

npData = np.around(np.array(dataDf)[:,1:7])
npData

tcDeathWoman = torch.from_numpy(deathWoman).to(device=cuda)
tcDeathMan = torch.from_numpy(deathMan).to(device=cuda)
tcNpData = torch.from_numpy(npData).to(device=cuda)
tcNpAgeData = torch.from_numpy(npAgeData).to(device=cuda)

pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

# 연령 분류 모델

In [23]:
def clusterAgeModel(rate,st,end):
    npData = np.zeros(100)
    
    for i in range(st,end):
        npData[i] = (rate**(i-end))-1  
        
    sumValue = np.sum(npData)
    npRateData = npData / sumValue
    return npRateData

In [21]:
%time clusterAgeModel(0.7,10,40)

CPU times: user 224 µs, sys: 0 ns, total: 224 µs
Wall time: 242 µs


array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.00060870e-01, 2.10040580e-01,
       1.47026377e-01, 1.02916435e-01, 7.20394754e-02, 5.04256038e-02,
       3.52958937e-02, 2.47050966e-02, 1.72915386e-02, 1.21020480e-02,
       8.46940462e-03, 5.92655423e-03, 4.14655897e-03, 2.90056228e-03,
       2.02836460e-03, 1.41782622e-03, 9.90449356e-04, 6.91285551e-04,
       4.81870888e-04, 3.35280624e-04, 2.32667439e-04, 1.60838209e-04,
       1.10557748e-04, 7.53614259e-05, 5.07240002e-05, 3.34778022e-05,
       2.14054636e-05, 1.29548266e-05, 7.03938064e-06, 2.89856850e-06,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

# 사망률 적용

In [24]:
def makeAlivePerson(data, deathRate):

    resultData = data - torch.round(data*deathRate)
    
    return resultData

# 은퇴율 적용

In [7]:
def makeWorkPerson(npData,tuningSet):
    valueList = np.zeros([2,100])
    c1 = tuningSet[2][0]/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = tuningSet[2][1]/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2
        
    torch.from_numpy
    result = npData - np.around(npData*valueList)
    
    return result

In [8]:
def makeWorkPerson1(tuningSet):
    valueList = np.zeros([2,100])
    c1 = 1/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = 1/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2

    
    return valueList

pd.DataFrame(makeWorkPerson1([[1.5,1.5],[25,25]]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.105457e-14,7.763643e-14,1.475092e-13,2.523184e-13,4.095322e-13,6.453529e-13,9.990839e-13,1.529680e-12,2.325575e-12,3.519417e-12,5.310181e-12,7.996325e-12,1.202554e-11,1.806937e-11,2.713511e-11,4.073372e-11,6.113163e-11,9.172850e-11,1.376238e-10,2.064668e-10,3.097312e-10,4.646278e-10,6.969728e-10,1.045490e-09,1.568266e-09,2.352431e-09,3.528677e-09,5.293047e-09,7.939601e-09,1.190943e-08,1.786418e-08,2.679630e-08,4.019448e-08,6.029176e-08,9.043766e-08,1.356565e-07,2.034848e-07,3.052273e-07,4.578409e-07,6.867614e-07,0.000001,0.000002,0.000002,0.000003,0.000005,0.000008,0.000012,0.000018,0.000026,0.00004,0.000059,0.000089,0.000134,0.0002,0.000301,0.000451,0.000677,0.001015,0.001522,0.002284,0.003425,0.005138,0.007707,0.011561,0.017342,0.026012,0.039018,0.058528,0.087791,0.131687,0.197531,0.296296,0.444444,0.666667
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.105457e-14,7.763643e-14,1.475092e-13,2.523184e-13,4.095322e-13,6.453529e-13,9.990839e-13,1.529680e-12,2.325575e-12,3.519417e-12,5.310181e-12,7.996325e-12,1.202554e-11,1.806937e-11,2.713511e-11,4.073372e-11,6.113163e-11,9.172850e-11,1.376238e-10,2.064668e-10,3.097312e-10,4.646278e-10,6.969728e-10,1.045490e-09,1.568266e-09,2.352431e-09,3.528677e-09,5.293047e-09,7.939601e-09,1.190943e-08,1.786418e-08,2.679630e-08,4.019448e-08,6.029176e-08,9.043766e-08,1.356565e-07,2.034848e-07,3.052273e-07,4.578409e-07,6.867614e-07,0.000001,0.000002,0.000002,0.000003,0.000005,0.000008,0.000012,0.000018,0.000026,0.00004,0.000059,0.000089,0.000134,0.0002,0.000301,0.000451,0.000677,0.001015,0.001522,0.002284,0.003425,0.005138,0.007707,0.011561,0.017342,0.026012,0.039018,0.058528,0.087791,0.131687,0.197531,0.296296,0.444444,0.666667


# 출신 연령별 분류모델

In [9]:
def makeArrayUseModel(tuningList):
    model1 = clusterAgeModel(tuningList[0][0], tuningList[1][0], tuningList[2][0]) # 의대 남
    model2 = clusterAgeModel(tuningList[0][1], tuningList[1][1], tuningList[2][1]) # 의대 여
    model3 = clusterAgeModel(tuningList[0][2], tuningList[1][2], tuningList[2][2]) # 의전원 남
    model4 = clusterAgeModel(tuningList[0][3], tuningList[1][3], tuningList[2][3]) # 의전원 여
    model5 = clusterAgeModel(tuningList[0][4], tuningList[1][4], tuningList[2][4]) # 재시험 남
    model6 = clusterAgeModel(tuningList[0][5], tuningList[1][5], tuningList[2][5]) # 재시험 여
    
    resultData =  np.array([model1, model2, model3, model4, model5, model6])
    
    return resultData

# 신규인원 생성

In [10]:
def makeNewPerson(npData, tuningSet):
    oldSize = 100
    yearSize = len(npData)
    modelSize = len(npData[0])
    
    #모델 적용 배열
    applyModelArray = np.zeros((yearSize,modelSize,oldSize))

    #신규인원 배열
    newPersonArray = np.zeros((yearSize,2,oldSize))

    
    modelAry = makeArrayUseModel(tuningSet)
    
    for i in range(yearSize):
        for j in range(modelSize):
            applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))

            if j%2==0:
                newPersonArray[i][0] += applyModelArray[i][j]
            else:
                newPersonArray[i][1] += applyModelArray[i][j]
    
    resultData = newPersonArray
    
    return resultData

# 최종인원 도출

In [11]:
def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    
    return dataArray

def makeResultPersonArray(newPersonArray, tuningSet):
    #누적인원 배열
    
    sizeArray = list(np.shape(newPersonArray))
    yearSize = sizeArray[0]
    resultPersonArray = np.zeros(sizeArray)

    for i in range(yearSize):
        if i>0:
            # 1살 올리기 / shiftData -> 2 x 100
            shiftData = shiftOld(resultPersonArray[i-1])
            # 사망률 적용 / aliveData -> 2 x 100
            aliveData = makeAlivePerson(shiftData, i)
            # 은퇴율 적용 / workData -> 2 x 100
            workData = makeWorkPerson(aliveData, tuningSet)
            # 최종 계산
            resultPersonArray[i] =  workData + newPersonArray[i]
        else:
            resultPersonArray[i] =  newPersonArray[i]
            
            
    return resultPersonArray

# 연령별 합산

In [12]:
def sumPeopleUseAge(npData):
    yearSize = len(npData)
    
    resultData = np.zeros([yearSize, 3, 12])
    
    for i in range(yearSize):
        for j in range(2):
            # 남/여 소계 계산
            resultData[i][j][0] = np.sum(npData[i][j])
            resultData[i][j][1] = np.sum(npData[i][j][0:30])
            resultData[i][j][2] = np.sum(npData[i][j][30:40])
            resultData[i][j][3] = np.sum(npData[i][j][40:50])
            resultData[i][j][4] = np.sum(npData[i][j][50:60])
            resultData[i][j][5] = np.sum(npData[i][j][60:])
            
            # 남/여 비율 계산
            for n in range(6):
                resultData[i][j][6+n] = resultData[i][j][n]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
        
        for j in range(6):
            # 합계 계산
            resultData[i][2][j] = resultData[i][0][j] + resultData[i][1][j]  
            # 합계 비율 계산
            resultData[i][2][6+j] = resultData[i][2][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            # 남/여 비율 계산
            resultData[i][0][6+j] = resultData[i][0][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            resultData[i][1][6+j] = resultData[i][1][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0


    return resultData

# 코스트 계산

In [13]:
def calculateCost(npData):   
    resultData = np.zeros([6,5,5])

    for i in range(6):
        manNumCost = npData[39-i][0][1:6] - npAgeData[5-i][8:13]
        womanNumCost = npData[39-i][1][1:6] - npAgeData[5-i][14:19]
        totalNumCost = npData[39-i][2][1:6] - npAgeData[5-i][2:7]
        
        manRateCost = npData[39-i][0][7:12] - npAgeData[5-i][26:31]
        womanRateCost = npData[39-i][1][7:12] - npAgeData[5-i][32:37]
        
        npCostArray = np.array([manNumCost, womanNumCost, totalNumCost, 
                    manRateCost, womanRateCost])

        resultData[i] = npCostArray

    return resultData

In [14]:
def getCost(npData, tuningSet):
    newPerson = makeNewPerson(npData, tuningSet[0])
    resultPerson = makeResultPersonArray(newPerson,tuningSet[1])
    # sumData -> 43 x 3 x 12
    sumData = sumPeopleUseAge(resultPerson)
    costData = calculateCost(sumData)
    
    return [sumData, costData]

In [15]:
## npData = np.around(np.array(dataDf)[:,1:7])
npData
#[0.9, 0.6, 0.1, 0.2, 39]]
tuningSetAgeRate = [[0.9,0.3,0.02,0.02,0.9,0.3],[26,26,28,28,27,27],[39, 39, 39, 39, 39, 39]]
tuningSetRetireRate = [[1.1, 1.1],[30, 30],[0.8, 0.8]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]
print(tuningSet)
%time data = getCost(npData,tuningSet)

[[[0.9, 0.3, 0.02, 0.02, 0.9, 0.3], [26, 26, 28, 28, 27, 27], [39, 39, 39, 39, 39, 39]], [[1.1, 1.1], [30, 30], [0.8, 0.8]]]
CPU times: user 38.4 ms, sys: 4.65 ms, total: 43.1 ms
Wall time: 40.6 ms


In [51]:
np.shape(data)
data[1][0]

array([[-4.67000000e+02,  2.37800000e+03,  5.30000000e+01,
         3.89200000e+03,  2.40000000e+01],
       [-8.04000000e+02,  1.43000000e+03,  1.59900000e+03,
         1.77800000e+03,  3.79000000e+02],
       [-1.27100000e+03,  3.80800000e+03,  1.65200000e+03,
         5.67000000e+03,  4.03000000e+02],
       [-9.67193538e-03,  2.44147423e-03, -2.19755300e-02,
         1.98828362e-02, -8.42710549e-03],
       [-1.10848089e-02,  2.53161360e-03,  9.49110965e-03,
         1.41859436e-02,  2.62640249e-03]])

In [128]:
dfResultCost = pd.DataFrame(data[1][0])
# dfResultCost.columns = ["남성오차", "여성오차", "남여총계오차","남비율오차", "여비율오차", "비율총계오차"]
# dfResultCost.index = ["2011", "2012", "2013","2014", "2015", "2016", "합계"]
dfResultCost

,0,1,2,3,4
0,-2117.000000,-1216.000000,-926.000000,1994.000000,-802.000000
1,-358.000000,1822.000000,939.000000,1240.000000,-70.000000
2,-2475.000000,606.000000,13.000000,3234.000000,-872.000000
3,-0.022182,-0.013657,-0.010797,0.019718,-0.008765
4,-0.003894,0.018253,0.009412,0.012688,-0.000775


In [132]:
dfResult = pd.DataFrame(data[0][0])
indexList = []
indexList.append('1977 남성')
indexList.append('1977 여성')
indexList.append('1977 합계')
    
for i in range(42):
    dfResult = pd.concat([dfResult,pd.DataFrame(data[0][i+1])])
    indexList.append(str(i+1978)+' 남성')
    indexList.append(str(i+1978)+' 여성')
    indexList.append(str(i+1978)+' 합계')

dfResult.columns = ['합계', '0~29', '30~39', '40~49', '50~59', '60~', '합계/비율', '0~29/비율','30~39/비율', '40~49/비율', '50~59/비율', '60~']
dfResult.index = indexList
dfResult

,합계,0~29,30~39,40~49,50~59,60~,합계/비율,0~29/비율,30~39/비율,40~49/비율,50~59/비율,60~
1977 남성,1026.0,590.0,436.0,0.0,0.0,0.0,0.855713,0.492077,0.363636,0.000000,0.000000,0.000000
1977 여성,173.0,172.0,1.0,0.0,0.0,0.0,0.144287,0.143453,0.000834,0.000000,0.000000,0.000000
1977 합계,1199.0,762.0,437.0,0.0,0.0,0.0,1.000000,0.635530,0.364470,0.000000,0.000000,0.000000
1978 남성,2061.0,1066.0,995.0,0.0,0.0,0.0,0.858750,0.444167,0.414583,0.000000,0.000000,0.000000
1978 여성,339.0,333.0,6.0,0.0,0.0,0.0,0.141250,0.138750,0.002500,0.000000,0.000000,0.000000
1978 합계,2400.0,1399.0,1001.0,0.0,0.0,0.0,1.000000,0.582917,0.417083,0.000000,0.000000,0.000000
1979 남성,3103.0,1406.0,1690.0,7.0,0.0,0.0,0.858843,0.389150,0.467755,0.001937,0.000000,0.000000
1979 여성,510.0,485.0,25.0,0.0,0.0,0.0,0.141157,0.134237,0.006919,0.000000,0.000000,0.000000
1979 합계,3613.0,1891.0,1715.0,7.0,0.0,0.0,1.000000,0.523388,0.474675,0.001937,0.000000,0.000000
1980 남성,4225.0,1630.0,2565.0,30.0,0.0,0.0,0.857520,0.330830,0.520601,0.006089,0.000000,0.000000


In [220]:
len(dfResult)

129

In [221]:
npAgeData = np.array(pd.read_excel('../../data/의사연령별분포.xlsx'))
pd.DataFrame(npAgeData).set_index(0)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011.0,82685.0,839.0,23488.0,27826.0,19466.0,11066.0,64371.0,561.0,14723.0,22244.0,16881.0,9962.0,18314.0,278.0,8765.0,5582.0,2585.0,1104.0,1.0,0.010147,0.284066,0.336530,0.235424,0.133833,0.778509,0.006785,0.178061,0.269021,0.204160,0.120481,0.221491,0.003362,0.106005,0.067509,0.031263,0.013352
2012.0,85073.0,1963.0,25100.0,27820.0,19381.0,10809.0,65989.0,1356.0,15836.0,22270.0,16796.0,9731.0,19084.0,607.0,9264.0,5550.0,2585.0,1078.0,1.0,0.023074,0.295041,0.327013,0.227816,0.127056,0.775675,0.015939,0.186146,0.261775,0.197430,0.114384,0.224325,0.007135,0.108895,0.065238,0.030386,0.012671
2013.0,88945.0,3459.0,27682.0,27933.0,19357.0,10514.0,68330.0,2297.0,17412.0,22366.0,16775.0,9480.0,20615.0,1162.0,10270.0,5567.0,2582.0,1034.0,1.0,0.038889,0.311226,0.314048,0.217629,0.118208,0.768228,0.025825,0.195761,0.251459,0.188600,0.106583,0.231772,0.013064,0.115465,0.062589,0.029029,0.011625
2014.0,91197.0,5078.0,28738.0,27892.0,19258.0,10231.0,69677.0,3258.0,18114.0,22374.0,16695.0,9236.0,21520.0,1820.0,10624.0,5518.0,2563.0,995.0,1.0,0.055682,0.315120,0.305843,0.211169,0.112186,0.764027,0.035725,0.198625,0.245337,0.183065,0.101275,0.235973,0.019957,0.116495,0.060506,0.028104,0.010910
2015.0,93484.0,6955.0,29528.0,27928.0,19203.0,9870.0,71066.0,4328.0,18808.0,22359.0,16656.0,8915.0,22418.0,2627.0,10720.0,5569.0,2547.0,955.0,1.0,0.074398,0.315862,0.298746,0.205415,0.105580,0.760194,0.046297,0.201190,0.239175,0.178170,0.095364,0.239806,0.028101,0.114672,0.059572,0.027245,0.010216
2016.0,96180.0,8795.0,30729.0,27938.0,19154.0,9564.0,72818.0,5272.0,19852.0,22420.0,16642.0,8632.0,23362.0,3523.0,10877.0,5518.0,2512.0,932.0,1.0,0.091443,0.319495,0.290476,0.199147,0.099439,0.757101,0.054814,0.206405,0.233105,0.173030,0.089748,0.242899,0.036629,0.113090,0.057372,0.026118,0.009690


In [222]:
npData = np.around(np.array(dataDf)[:,1:7])
npData

tuningSetAgeRate = [[0.7,0.7,0.9,0.9,0.9,0.9],[26,26,28,28,27,27],[40, 40, 40, 40, 40, 40]]
tuningSetRetireRate = [[1.1, 1.1],[25, 25],[1, 1]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]
print(tuningSet)
data = getCost(npData,tuningSet)



tuningList = range(0.1, 1.0, 0.1)



def makeTuningSet(tuningSetAgeRate):
    
    
    return 





[[[0.7, 0.7, 0.9, 0.9, 0.9, 0.9], [26, 26, 28, 28, 27, 27], [40, 40, 40, 40, 40, 40]], [[1.1, 1.1], [25, 25], [1, 1]]]


TypeError: 'float' object cannot be interpreted as an integer

In [86]:
def test():
    npData = np.around(np.array(dataDf)[:,1:7])
    npData

    meanCost = np.array([])
    meanParameter = np.array([])
    resultData = []

    for h in np.arange(0.1,1,0.1):
        for i in np.arange(0.1,1,0.1):
            for j in np.arange(0.1,1,0.1):
                for k in np.arange(0.1,1,0.1):
                    for m in np.arange(30,40,1):
                        tuningSetAgeRate = [[h,i,j,k,h,i],[26,26,28,28,27,27],[m, m, m, m, m, m]]
                        tuningSetRetireRate = [[0,0],[0, 0],[0, 0]]
                        tuningSet = [tuningSetAgeRate, tuningSetRetireRate]
                        data = getCost(npData,tuningSet)
                        resultData.append([[h,i,j,k,m ] ,data[1]])
    return resultData
                
%time resultData = np.array(test())

CPU times: user 22min 9s, sys: 2.56 s, total: 22min 12s
Wall time: 22min 13s


TypeError: Mismatch between array dtype ('object') and format specifier ('%f,%f')

In [89]:
saveData = resultData

In [104]:
dfResultCost = pd.DataFrame(saveData[0][1][0])
dfResultCost.columns = ["20대", "30대", "40대","50대", "60~"]
dfResultCost.index = ["남", "여", "남여","남비율", "여비율"]
dfResultCost

,20대,30대,40대,50대,60~
남,574.000000,3746.000000,223.000000,3087.000000,-1725.000000
여,-174.000000,2025.000000,1030.000000,1463.000000,70.000000
남여,400.000000,5771.000000,1253.000000,4550.000000,-1655.000000
남비율,0.000079,0.015175,-0.020492,0.012221,-0.024893
여비율,-0.005183,0.008057,0.004113,0.011207,-0.000282


In [111]:
cost = 0
cost += np.sum(np.abs(saveData[0][1][0][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][1][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][2][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][3][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][4][3:5,:]))
cost += np.sum(np.abs(saveData[0][1][5][3:5,:]))
cost

1.3705501773291702

array([[[ 4.84600000e+03,  5.93200000e+03,  4.05000000e+02,
         -2.64000000e+02, -6.14600000e+03],
        [ 2.53700000e+03,  9.33000000e+02,  2.20000000e+02,
          1.51000000e+02, -4.90000000e+02],
        [ 7.38300000e+03,  6.86500000e+03,  6.25000000e+02,
         -1.13000000e+02, -6.63600000e+03],
        [ 4.77618820e-02,  4.47272989e-02, -1.68730596e-02,
         -1.85041974e-02, -7.22360178e-02],
        [ 2.50426604e-02, -5.18653299e-05, -2.97185497e-03,
         -8.73945548e-04, -6.02090073e-03]],

       [[ 6.37600000e+03,  7.28800000e+03,  7.54000000e+02,
         -1.65200000e+03, -7.45900000e+03],
        [ 3.35900000e+03,  1.12000000e+03,  1.46000000e+02,
         -2.62000000e+02, -7.07000000e+02],
        [ 9.73500000e+03,  8.40800000e+03,  9.00000000e+02,
         -1.91400000e+03, -8.16600000e+03],
        [ 6.62845770e-02,  5.97598075e-02, -1.69327805e-02,
         -3.63857358e-02, -9.02231575e-02],
        [ 3.50402884e-02,  1.53108043e-03, -4.66554296e-03,
  

In [236]:
npData = np.around(np.array(dataDf)[:,1:7])
npData

meanCost = np.array([])
meanParameter = np.array([])

In [94]:
range(len(resultData))

range(0, 65610)

In [122]:
minCost = [1000000,1000000,[]]
for i in range(65610):
    cost1 = 0
    cost1 += np.sum(np.abs(saveData[i][1][0][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][1][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][2][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][3][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][4][3:5,:]))
    cost1 += np.sum(np.abs(saveData[i][1][5][3:5,:]))
    cost1
    
    cost2 = 0
    cost2 += np.sum(np.abs(saveData[i][1][0][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][1][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][2][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][3][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][4][2,:]))
    cost2 += np.sum(np.abs(saveData[i][1][5][2,:]))
    cost2

    if cost1<minCost[0]:
        if cost2<minCost[1]:
            minCost = [cost1,cost2,resultData[i][0]]
        
minCost

[0.8681019207294333, 84264.0, [0.9, 0.30000000000000004, 0.1, 0.1, 39]]

[66763.0, [0.9, 0.30000000000000004, 1.1, 0.7000000000000001]]
[66763.0, [0.9, 0.30000000000000004, 1.1, 1.1]]
[70727.0, [0.9, 0.9, 1.1, 0.9]]

[65443.0, [0.9, 0.9, 1.1, 0.30000000000000004]]
[65211.0, [0.9, 0.1, 1.1, 0.7999999999999999]]